In [5]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from pyclim_noresm.general_util_funcs import global_avg
import matplotlib as mpl
from workflow.scripts.plotting_tools import global_map
import xesmf as xe
import cf_xarray as cfxr

In [6]:
paths = snakemake.input.paths
grid_area = snakemake.input.areacello

In [31]:
dsets = []
budgets = {}
ds_out = xe.util.grid_global(3, 2,cf=True)
for path, ga_path in zip(paths, grid_area):
    ds = xr.open_dataset(path)
    ga = xr.open_dataset(ga_path)
    yearly_budget = ds[ds.variable_id].resample(time='Y').mean()*365*24*60*60
    
    yearly_budget = yearly_budget.mean(dim='time')
    yearly_budget = yearly_budget*ga['cell_area']
    yearly_budget = yearly_budget.sum(dim=['lon','lat']).values
    budgets[ds.source_id]=yearly_budget
    ds = ds.groupby('time.month').mean(dim='time', keep_attrs=True).mean('month', keep_attrs=True)
    regridder= xe.Regridder(ds,ds_out,'conservative')
    
    #ds = ds.rename({ds.variable_id:ds.source_id})
    ds = regridder(ds,keep_attrs=True)
    ds[ds.variable_id] = ds[ds.variable_id]*1000
    print(ds.emidust.max())
    dsets.append(ds)



In [36]:
fig,ax = plt.subplot_mosaic([['a)','b)','c)'],['d)','e)','f)'],['g)','h)','i)']], 
                            subplot_kw={'projection':ccrs.PlateCarree()}, figsize=(18,10), sharex=True, sharey=True)
cmap = mpl.cm.YlOrBr
norm = mpl.colors.LogNorm(vmin=1e-8, vmax=1e-4)
for k,ds, sids in zip(ax.keys(), dsets, budgets.keys()):
    ds[ds.variable_id].plot.pcolormesh(ax=ax[k], norm=norm, cmap=cmap, add_colorbar=False,vmin=1e-8, vmax=1e-4)
    ax[k].coastlines()
    global_map(ax[k])
    ax[k].set_title('{} [Tg/year]    {}'.format(round(budgets[sids]/1000000000),ds.source_id))
    
cax = fig.add_axes([0.94,0.2,0.02,0.62])
fig.colorbar(mpl.cm.ScalarMappable(norm, cmap=cmap), cax=cax, extend='neither',
             label='{} [g m-2 s-1]'.format(ds.variable_id))
plt.savefig(snakemake.output.outpath, bbox_inches='tight', facecolor='white')